In [376]:
import pandas as pd

# Cleaning &  Preparing the data to classify

In [377]:
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize 
import re

stemmer = SnowballStemmer("english")
def format_sentence(sent):
    sent = sent.lower()
    sent = re.sub('\s+', ' ', sent) #exstra space
    sent = ' '.join([stemmer.stem(word) for word in re.split('\s+', sent)]) #Stemming
    sent = re.sub("[^0-9A-Za-z ]", "", sent) #Punctuations 
    return {word: (word in word_tokenize(sent)) for word in word_tokenize(sent)}

format_sentence('this’s is a interesting boy   @#!$%^&')

{'this': True, 'is': True, 'a': True, 'interest': True, 'boy': True}

In [378]:
data = pd.read_csv('data.csv', encoding= 'unicode_escape').values.tolist()

## Reading the data

In [379]:
pos_data = []
neg_data = []
neu_data = []

for rew in range(len(data)):
    record = data[rew]
    line = str(record[0])   
    category = str(record[1])
    if(category == 'positive'):
        pos_data.append([format_sentence(line), 'pos'])
    elif(category == 'negative'):
        neg_data.append([format_sentence(line), 'neg'])
    elif(category == 'neutral'):
        neu_data.append([format_sentence(line), 'neutral'])
    

In [380]:
print(len(pos_data), len(neg_data), len(neu_data))

1852 860 3130


In [381]:
posit_data = []
with open('extra.pos.txt', encoding='latin-1') as f:
    for line in f:
        posit_data.append([format_sentence(line), 'pos'])

In [382]:
negat_data = []
with open('extra.neg.txt', encoding='latin-1') as f:
    for line in f:
        negat_data.append([format_sentence(line), 'neg'])

In [383]:
pos_data.extend(posit_data)
neg_data.extend(negat_data)

In [384]:
print(len(pos_data), len(neg_data), len(neu_data))

7183 6191 3130


In [390]:
training_data = pos_data[:6700] + neg_data[:5500] + neu_data[2000:] 
testing_data  = pos_data[6700:] + neg_data[5500:] + neu_data[:2000]

In [391]:
print(len(training_data), len(testing_data))

13330 3174


In [392]:
from nltk.classify import NaiveBayesClassifier
model = NaiveBayesClassifier.train(training_data)
#model.show_most_informative_features()

In [393]:
print(model.classify(format_sentence('this is a nice boy!')))
print(model.classify(format_sentence('this is a bad boy!')))
print(model.classify(format_sentence('happyness')))
print(model.classify(format_sentence('what is ur name?')))
print(model.classify(format_sentence('According to the Finnish-Russian Chamber of Commerce.')))
print(model.classify(format_sentence('Viking Line has canceled some services.')))
print(model.classify(format_sentence('my name is Mohamed Khalid.')))

pos
neg
pos
neg
neutral
neutral
neutral


In [394]:
from nltk.classify.util import accuracy
accuracy(model, testing_data)

0.8793320730938878